## ISRC Python Workshop 2: Web Scraping (Mar. 09th 2017)

## Introduction

<img src="https://upload.wikimedia.org/wikipedia/commons/1/10/CSS3_and_HTML5_logos_and_wordmarks.svg" width="526" height="340">

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Text-xml.svg/2000px-Text-xml.svg.png" width="200" height="150">

<img src="http://career.guru99.com/wp-content/uploads/2014/12/json-logo1.png" width="400" height="200">

### Parsing Contents

Examples from <a href="https://www.w3schools.com/html/tryit.asp?filename=tryhtml_basic_document" target="blank_">w3schools</a>.

```html
<!DOCTYPE html>
<html>
<body>

<h1>My First Heading</h1>

<p>My first paragraph.</p>

</body>
</html>
```